# Page Rank

### Intro
As we walk through the algorithm, I'm going to try to explain using plain English. It's not every day that I get to explain the gap between matrix operations and imagining that network in our minds. How do they connect? Well, I'm going to show you!

In [16]:
#import statements here
import numpy as np
import pandas as pd


### Setup
Here is where we begin setting up the pieces of our problem, so to speak. I'm going to give a basic definition for each of the pieces.
* **ALPHA**: the probability of following the network naturally, *without* a "teleport"
* **EPSILON**: when we do any kind of stochastic processes, we won't always "converge" perfectly. So that means there's a little teeny bit of wiggle room for us to count iterations as giving us the "same" answer. The wiggle room is this epsilon (it's small!)
* **MAX_ITER**: we are running our algorithm by iterating, rather than any closed-form solution. The number of times we run the update is here (For the test network it can be super small. It converges in 15 steps! But for larger networks, it's a little bit of trial and error seeing how long we need it to run until it converges)
* **pi_star**: The whole objective of this operation! We want the vector of weights that tell us the importance of each of the pages. We start this vector with uniform weights, and update as we iterate. At the end, we should have a distribution of weights that ranks our pages.
* **Z** : Z is an adjacency matrix. It is also "the network itself". How do we think about that? How can a matrix be the same as a network? Well, it holds the transition probabilities between nodes. We can traverse it just like we can a node-and-edge network structure. Also an important concept to keep in mind is that while in the network example we picture one random surfer walking around, in matrix operations we can imagine infinite walkers *at the same time*.

In [17]:
ALPHA = .85 # chosen "because it works" and is not that unreasonable
EPSILON = 0.00001 # chosen "because it works" and is small
MAX_ITER = 100
# initial start vector (start with prior weights at 1/n because it's reasonable to have a uniform prior)
pi_star = np.array([]) # the influence vector weights we will calculate!



articles =[] # number of papers from our sample out of each journals looked at

Z = np.array([
                [1,0,2,0,4,3],
                [3,0,1,1,0,0],
                [2,0,4,0,1,0],
                [0,0,1,0,0,1],
                [8,0,3,0,5,2],
                [0,0,0,0,0,0]
            
            ]) #original matrix

    
# remove self-citers, by setting the diagonal to 0
np.fill_diagonal(Z,0)
print(Z)




[[0 0 2 0 4 3]
 [3 0 1 1 0 0]
 [2 0 0 0 1 0]
 [0 0 1 0 0 1]
 [8 0 3 0 0 2]
 [0 0 0 0 0 0]]


In [18]:
def normalize(M):
    normed = M/M.sum(axis=0) #normalization
    normed = np.nan_to_num(normed) #the way I computed the normed actually results in some NaNs when I divide by 0...
    return normed

H = normalize(Z) #normalized matrix of journal citing, also with self-cite removed (aka set to 0)
print(H)



[[0.         0.         0.28571429 0.         0.8        0.5       ]
 [0.23076923 0.         0.14285714 1.         0.         0.        ]
 [0.15384615 0.         0.         0.         0.2        0.        ]
 [0.         0.         0.14285714 0.         0.         0.16666667]
 [0.61538462 0.         0.42857143 0.         0.         0.33333333]
 [0.         0.         0.         0.         0.         0.        ]]


C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [19]:
def find_dangles(M):
    
    non_z = np.count_nonzero(M,axis=0) #if the number of non-zeroes is 0, it's a dangling node!
    dangles = [1 if nz==0 else 0 for nz in non_z]
    dangles = np.array(dangles) # don't cite other journals. 0 or 1 if is_dangle
    return dangles
    
dangles = np.array(find_dangles(H))
print(dangles)

[0 1 0 0 0 0]


In [20]:
a_start = [3,2,5,1,2,1]
a_sum = sum(a_start)
articles = np.array([v/a_sum for v in a_start]).T
print(articles)
# TODO: add in assert statements for shape along the way

# this commented out code actually makes H' ("H prime") which we do not need for the iterative approach
'''for i in dangles:
    if i==1:
        print(H[:,i]) #finds the column that is a dangler
        H[:,i]=articles #replace with "base probabilities" aka the article vector
print(H)'''


[0.21428571 0.14285714 0.35714286 0.07142857 0.14285714 0.07142857]


'for i in dangles:\n    if i==1:\n        print(H[:,i]) #finds the column that is a dangler\n        H[:,i]=articles #replace with "base probabilities" aka the article vector\nprint(H)'

In [21]:
#defining start values
#need to match shape (number of rows)
pi_star= np.array(np.full(H.shape[0],1/H.shape[0]))
print(pi_star.shape)


(6,)


In [22]:
import copy #just to be safe?
# check if the previous vector is the same as the current one, with some tolerance epsilon
def check_converge(v0,v1,e):
    if (abs(v1 - v0).sum()) < e:
        return True
    else:
        return False
    
#π(k+1) = α H π(k) + [α d.π(k) + (1 − α)]a
# let's do our "power up" or "random walk" here
print(pi_star)
dangles = dangles.T
for i in range(MAX_ITER):
    previous = copy.deepcopy(pi_star)
    dampening = ((ALPHA*dangles.dot(previous)) + (1-ALPHA))*articles
    #print(dampening)
    
    pi_star = (ALPHA * H.dot(previous)) +dampening

    if check_converge(previous, pi_star, EPSILON):
        print(i) #number it takes to converge
        break #stop iterating, we are done!

pi_star=[v/sum(pi_star) for v in pi_star]    #normalize just in case even though it's totally already normalized
print(pi_star)  


    



[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
17
[0.3040213756115484, 0.1636030408810989, 0.1897961587420407, 0.04661906101548591, 0.2753130900296041, 0.020647273720221997]


In [23]:
eigen_fac = 100*(H.dot(pi_star)/sum(H.dot(pi_star))) 
print(eigen_fac)

[34.05100649 17.20374224 12.17545523  3.6531636  32.91663244  0.        ]


## So we made it through the test case!
What follows is testing on an actual dataset (an actually small one, comparatively speaking!)
This highlights what is particularly difficult about Machine Learning (research or just applying it). Every sub-optimal piece of your code will be exaggerated when dealing with millions of nodes.

### Weirdly, one of my issues was creating the adjacency matrix itself.
I was so confused about how to turn the edge list into a proper adjacency matrix! Is there some numpy command to do it? Otherwise I did it in such a weird way. Held on to dictionaries, etc. For the record, we are dealing with a VERY SPARSE adjacency matrix. They warn you about those. Like the boogie man of matrices. SPARSITY = lots of unnecessary traversing for little gain! * cry *

In [24]:
# read in from the txt file instead, and convert to adjacency matrix
M = {}
import csv
with open('links.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i=0
    nodes = {}
    for row in csv_reader:
          
        if int(row[0]) in M:
            M[int(row[0])][int(row[1])]=int(row[2])
        else:
            M[int(row[0])]={int(row[1]):int(row[2])}
            nodes[int(row[0])]=i
            i+=1
        
        #we know the dangling nodes by the ones that never appear as "cited" but never "citers".
        
        
        
        



In [25]:

for k,v in M.items():
    for k2,v2 in v.items():
        if k2 not in nodes:
            nodes[k2]=len(nodes)
print(nodes)

print(len(nodes))         




{758: 0, 42: 1, 5584: 2, 8189: 3, 5623: 4, 2905: 5, 7183: 6, 9745: 7, 9719: 8, 2975: 9, 5830: 10, 5518: 11, 6965: 12, 3710: 13, 8770: 14, 4762: 15, 4076: 16, 3904: 17, 2722: 18, 4750: 19, 2501: 20, 7176: 21, 3037: 22, 5421: 23, 1652: 24, 6921: 25, 7159: 26, 6093: 27, 6146: 28, 7096: 29, 6514: 30, 5059: 31, 7543: 32, 6815: 33, 5157: 34, 216: 35, 5139: 36, 2747: 37, 4439: 38, 6690: 39, 6791: 40, 2767: 41, 4685: 42, 5448: 43, 6701: 44, 10313: 45, 5917: 46, 3260: 47, 190: 48, 9732: 49, 2756: 50, 1340: 51, 5334: 52, 5203: 53, 2873: 54, 6901: 55, 7620: 56, 6240: 57, 1687: 58, 1523: 59, 9591: 60, 3837: 61, 3580: 62, 4339: 63, 359: 64, 6697: 65, 4088: 66, 5253: 67, 10017: 68, 2985: 69, 7409: 70, 7979: 71, 6235: 72, 3909: 73, 1134: 74, 405: 75, 10114: 76, 3202: 77, 4728: 78, 3299: 79, 3657: 80, 1503: 81, 6414: 82, 2764: 83, 6495: 84, 8679: 85, 7287: 86, 1421: 87, 3812: 88, 6112: 89, 401: 90, 5481: 91, 4519: 92, 9336: 93, 3197: 94, 3945: 95, 2461: 96, 451: 97, 5958: 98, 7163: 99, 5610: 100, 6722

### Well, I finally got the adjacency matrix

In [26]:
matrix = np.zeros(shape=[len(nodes),len(nodes)])
for k,v in M.items():
    for k2,v2 in v.items():
        #print(k,k2)
        matrix[nodes[k]][nodes[k2]]=v2
        
        



In [27]:
print(matrix) #you can see that Journal 758 cites itself 150 times. We will get rid of the self-citing though.

[[150.   0.   0. ...   0.   0.   0.]
 [  0. 186.   0. ...   0.   0.   0.]
 [  1.   0. 558. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


### Reformatted everything to belong in a single function
Here's the whopping "pagerank" function. I run it on the test case (labelled toy_problem) just to make sure the function is passing everything properly. You can see it print out the following influence vector:
    
    0.30402138 0.16360304 0.18979616 0.04661906 0.27531309 0.02064727
    
and the eigen factor:
     
     34.05100649 17.20374224 12.17545523  3.6531636  32.91663244  0.
All correct on the test case!

### So what goes wrong with the real data?
I think something is going wrong with overflows or the way I'm multiplying matrices or something. I really don't know but I'm getting a big vector of NaNs and ran out of time to dig into that bug. Let me know if anything is apparent to you. It runs fast enough, but isn't doing the right computation on the network. I assume I have some shapes off or read the matrix in weird. Again, if there's some magic numpy function to convert edgelists to adjacency matrices, maybe I would have been better off. Apologies.

In [28]:
#let's reformat everything from above into a packaged function!

def pagerank(matrix,ALPHA=.85,EPSILON=.00001,MAX_ITER=1000,articles=False):
    influence_vector = np.array([]) # the influence vector weights we will calculate!
    # remove self-citers, by setting the diagonal to 0
    np.fill_diagonal(matrix,0)
    matrix_ = normalize(matrix)
    dangles = np.array(find_dangles(matrix_))
    print(len(dangles))
    
    if articles:
        #hardcoding this so hard, sorry!
        a_start = [3,2,5,1,2,1]
        a_sum = sum(a_start)
        article_vector = np.array([v/a_sum for v in a_start]).T
    else: 
        article_vector = np.ones(shape=[len(matrix)])
        
    
        
    #we don't need to take into account articles, assuming a uniform prior for the teleport
    
    influence_vector= np.array(np.full(matrix_.shape[0],1/matrix_.shape[0]),dtype=np.float64)
    for i in range(MAX_ITER):
        previous = copy.deepcopy(influence_vector)
    
        dampening = ((ALPHA*dangles.dot(previous)) + (1-ALPHA))*article_vector
            
        #print(dampening)

        influence_vector = (ALPHA * matrix_.dot(previous)) +dampening
        influence_vector = np.nan_to_num(influence_vector)
        

        if check_converge(previous, influence_vector, EPSILON):
            print("CONVERGE: " + str(i)) #number it takes to converge
            print(influence_vector)
            break #stop iterating, we are done!

   
    eigen_fac = 100*(matrix_.dot(influence_vector)/(sum(matrix_.dot(influence_vector)))) 
    eigen_fac2= np.nan_to_num(eigen_fac)
    
    return(eigen_fac2)

toy_problem = pagerank(Z,articles=True)
print(toy_problem)

real_answer = pagerank(matrix)
print(np.sort(real_answer))

6
CONVERGE: 17
[0.30402138 0.16360304 0.18979616 0.04661906 0.27531309 0.02064727]
[34.05100649 17.20374224 12.17545523  3.6531636  32.91663244  0.        ]


C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


10748


C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


CONVERGE: 185
[1.79769313e+308 1.79769313e+308 1.79769313e+308 ... 1.79769313e+308
 1.79769313e+308 1.79769313e+308]


C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide


[0. 0. 0. ... 0. 0. 0.]


### Apologies, I ran out of time!

So obviously I did something wrong with reading in my matrix. I've tried to fix it with some scipy things below, but my pagerank algorithm doesn't want to run on that. If you see the issue please please let me know, *because I think I was really close!*

In [29]:
#pd.DataFrame(matrix).to_csv("matrix.csv",header=None)

# import links
links = np.array(np.loadtxt(open("links.txt", "r"), delimiter=",",dtype=int))
print(links)


[[ 758 1476    5]
 [ 758  758  150]
 [ 758 5938    3]
 ...
 [9742 7940    1]
 [9742 7744    1]
 [9742 5130    0]]


### So at least I figured out you can read in the matrix like this.
But now I'm still having issues with processing the matrix because of how I wrote my algorithm. Hopefully not too many points get taken off, but yes I didn't finish the assignment. If you have time, please let me know where I was going wrong! :D 

In [30]:
import scipy.sparse as sparse
arr = np.array(links)
shape = tuple(arr.max(axis=0)[:2]+1)
coo = sparse.coo_matrix((arr[:, 2], (arr[:, 0], arr[:, 1])), shape=shape,
                        dtype=arr.dtype)

print(coo)

coo2=np.array(coo.todense())
print(coo2)
print(coo2.shape)

val = pagerank(coo2)

  (758, 1476)	5
  (758, 758)	150
  (758, 5938)	3
  (758, 4972)	13
  (758, 2416)	0
  (758, 7067)	1
  (758, 4543)	0
  (758, 2722)	1
  (758, 2249)	1
  (758, 7531)	1
  (758, 5046)	0
  (758, 5250)	2
  (758, 1611)	2
  (758, 1441)	0
  (758, 4032)	7
  (758, 1419)	16
  (758, 630)	42
  (758, 6967)	2
  (758, 3683)	0
  (758, 6732)	121
  (758, 7332)	96
  (758, 604)	1
  (758, 4341)	7
  (758, 937)	3
  (758, 2728)	4
  :	:
  (9742, 5624)	0
  (9742, 2602)	0
  (9742, 4358)	1
  (9742, 5193)	0
  (9742, 4104)	0
  (9742, 8848)	0
  (9742, 1207)	3
  (9742, 10705)	1
  (9742, 8747)	0
  (9742, 4992)	0
  (9742, 3491)	1
  (9742, 2519)	2
  (9742, 4198)	1
  (9742, 7268)	0
  (9742, 4408)	0
  (9742, 8269)	1
  (9742, 3255)	6
  (9742, 1172)	1
  (9742, 7399)	0
  (9742, 885)	1
  (9742, 5163)	0
  (9742, 1318)	0
  (9742, 7940)	1
  (9742, 7744)	1
  (9742, 5130)	0
[[  34    0    0 ...    0    0    0]
 [   0   21    0 ...    0    0    0]
 [   0    0 1594 ...    0    0    0]
 ...
 [   0    0    0 ...   20    0    0]
 [   0    0 

C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


10748


C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


CONVERGE: 185
[1.79769313e+308 1.79769313e+308 1.79769313e+308 ... 1.79769313e+308
 1.79769313e+308 1.79769313e+308]


C:\Users\jennareg\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide
